## Machine Learning for Neuroscience, <br>Department of Brain Sciences, Faculty of Medicine, <br> Imperial College London
### Contributors: Payam Barnaghi, Anastasia Gailly de Taurines, Antigone Fogel, Iona Biggart, Nan Fletcher-Lloyd
**Spring 2026**

In [ ]:
import torch
from torch import nn

The code is adapted from: https://d2l.ai/chapter_convolutional-neural-networks/conv-layer.html

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.imshow(X, cmap=plt.cm.hot)
plt.show()

Defintion of the 2D filter mapping and the convolution funciton - these allow to slide a filter across the image block

In [ ]:
def corr2d(X, K):  
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

The filter

In [ ]:
K = torch.tensor([[1.0, -1.0]])

In [ ]:
plt.imshow(K, cmap=plt.cm.hot)
plt.show()

Now apply the filter to the original image/data

In [ ]:
Y = corr2d(X, K)
Y

In [ ]:
plt.imshow(Y, cmap=plt.cm.hot)
plt.show()


In the above example we used a static kernel. But in large networks this will not be possible. In CNN training we can use the training data to learn a kernel (that's one beauty of learning via multiple iteration). We can start from a random kernel state and then learn it through the training phase. Now we can explore a kernel can be leartned. 

In [ ]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
# adapted from https://d2l.ai/chapter_convolutional-neural-networks/conv-layer.html

conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

In [ ]:
X

In [ ]:
Y

In [ ]:
conv2d.weight.data

This is pretty cool! 

In a few iterations we almost learned the same edge detection kernel that we have hard coded intially [1.0, -1.0] (see above). 

Excercise: Try to change the learning rate (lr) and also the number of iterations (for i in range(10)) (e.g. change to 20) and see how the results change.